In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
sample = pd.read_csv('../input/sample_submission.csv')

In [ ]:
# # memory size finding
# train_size_mb = train.memory_usage().sum() / 1024 / 1024
# print("Test memory size: %.2f MB" % train_size_mb)
# # Test memory size: 1879.24 MB


In [ ]:
np.unique(train['BldgType'], return_counts=True)

In [ ]:
train.shape

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.scatter(train['SalePrice'], train['Neighborhood']);

### Analysis of Handling Categorical Data

In [ ]:
ex = [train['SalePrice'][i] for i in range(train.shape[0]) if train['KitchenQual'][i] == 'Ex']
ta = [train['SalePrice'][i] for i in range(train.shape[0]) if train['KitchenQual'][i] == 'TA']
fa = [train['SalePrice'][i] for i in range(train.shape[0]) if train['KitchenQual'][i] == 'Fa']
gd = [train['SalePrice'][i] for i in range(train.shape[0]) if train['KitchenQual'][i] == 'Gd']

In [ ]:
ex_mean = np.mean(ex)
ta_mean = np.mean(ta)
fs_mean = np.mean(fa)
gd_mean = np.mean(gd)

ex_std = np.std(ex)
ta_std = np.std(ta)
fs_std = np.std(fa)
gd_std = np.std(gd)

In [ ]:
print(ex_mean, ex_std)
print(ta_mean, ta_std)
print(fs_mean, fs_std)
print(gd_mean, gd_std)

### seperate the object type feature from numerical type 

In [ ]:
train_otype = train.select_dtypes(include=['object']).copy()


In [ ]:
train_otype.head()

In [ ]:
print(train.shape)
print(train_otype.shape)

### get the null values

In [ ]:
train_otype.isnull().values.sum()

In [ ]:
# col wise null values
train_otype.isnull().sum()

In [ ]:
##It seems like 14 column has null values. 
# We can do a mode imputation for those null values. 
#The function fillna() is handy for such operations.

train_otype = train_otype.fillna(train_otype['Alley'])

In [ ]:
train_otype.isnull().values.sum()

In [ ]:
train = pd.get_dummies(train)
test = pd.get_dummies(test)


In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
test[:3]

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sklearn
import sklearn.preprocessing as pp
import sklearn.model_selection as ms
import sklearn.metrics as metrics
import json

### Load Dataset

In [ ]:
df = pd.read_csv('../input/train.csv')
df = df.set_index('Id')

sdf = pd.read_csv('../input/test.csv')
sdf = sdf.set_index('Id')
df.head()


In [ ]:
price = df.SalePrice
print("Average sale price: " + "${:,.0f}".format(price.mean()))

### Combine test and train for preprocessing

In [ ]:
df = df.drop('SalePrice', axis=1)
all_df = df.append(sdf)
all_df.shape


In [ ]:
all_features = 'MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition'.split(',')
numeric_features = ['LotFrontage', 'LotArea', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','LowQualFinSF','GrLivArea','BsmtFullBath','BsmtHalfBath','FullBath','HalfBath','BedroomAbvGr','KitchenAbvGr','TotRmsAbvGrd','TotalBsmtSF','Fireplaces', 'GarageCars', 'GarageArea','WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal']
categorical_features = [f for f in all_features if not(f in numeric_features)]

(len(all_features), len(categorical_features), len(numeric_features))


In [ ]:
numeric_df = all_df[numeric_features]
numeric_df.shape


### 
- Impute Only impute for numeric variables, categorical will have a NA column


In [ ]:
X = numeric_df.as_matrix()

# Impute missing

imp = pp.Imputer(missing_values='NaN', strategy='most_frequent', axis=0)
imp = imp.fit(X)
X = imp.transform(X)
X.shape

In [ ]:
scaler = pp.StandardScaler()
scaler.fit(X)
X = scaler.transform(X)
X[0, :]

### Expand Categorical Cols

In [ ]:
def process_categorical(ndf, df, categorical_features):
    for f in categorical_features:
        new_cols = pd.DataFrame(pd.get_dummies(df[f]))
        new_cols.index = df.index
        ndf = pd.merge(ndf, new_cols, how = 'inner', left_index=True, right_index=True)
    return ndf

numeric_df = pd.DataFrame(X)
numeric_df.index = all_df.index
combined_df = process_categorical(numeric_df, all_df, categorical_features)
combined_df.head()



In [ ]:


X = combined_df.values
X.shape



In [ ]:
#PCA
from sklearn.decomposition import PCA

test_n = df.shape[0]
print(test_n)
pca = PCA()
pca.fit(X[:test_n,:], price)
X = pca.transform(X)
X.shape

In [ ]:
X_train = X[:test_n,:]
X_train, X_val, y_train, y_val = ms.train_test_split(X_train, price, test_size=0.3, random_state=0)
X_test = X[test_n:,:]

(X_train.shape, X_val.shape, X_test.shape)

### Linear Regression Model

In [ ]:
from sklearn import linear_model

lr = linear_model.LinearRegression()
lr.fit(X_train, y_train)

def print_score(alg, score, params):
    print('%s score is %f with params %s' % (alg, score, json.dumps(params)))


### Lasso Regression

In [ ]:
for a in np.arange(151., 152., 0.1):
    lasso = linear_model.Lasso(alpha=a, max_iter=2000)
    lasso.fit(X_train, y_train)
    print_score('Lasso', lasso.score(X_val, y_val), {'alpha': a})



In [ ]:
lasso = linear_model.Lasso(alpha=151.7, max_iter=2000)
lasso.fit(X_train, y_train)

### Ridge Regression


In [ ]:
for a in np.arange(25., 26., 0.05):
    ridge = linear_model.Ridge(alpha=a, max_iter=2000)
    ridge.fit(X_train, y_train)
    print_score('Lasso', ridge.score(X_val, y_val), {'alpha': a})

In [ ]:
ridge = linear_model.Ridge(alpha=25.5, max_iter=2000)
ridge.fit(X_train, y_train)

### Gradient Boosting


In [ ]:
import xgboost as xgb
params = {'eval_metric':'rmse'}
xm = xgb.DMatrix(X_train, label=y_train)
xmodel = xgb.train(params, xm)
xg_y_pred = xmodel.predict(xgb.DMatrix(X_val))

In [ ]:
print('XGBoost score is %f' % metrics.r2_score(y_val, xg_y_pred))
print('Ridge score is %f' % ridge.score(X_val, y_val))
print('Lasso score is %f' % lasso.score(X_val, y_val))
print('Linear Regression score is %f' % lr.score(X_val, y_val))

best = ridge

In [ ]:
best.fit(X[:test_n, :], price)
y_submit = best.predict(X_test)
y_submit[y_submit < 0] = 1.
sdf['SalePrice'] = y_submit
sdf.to_csv('submission.csv', columns = ['SalePrice'])

xmodel = xgb.train(params, xgb.DMatrix(X[:test_n, :], label=price))
y_submit = xmodel.predict(xgb.DMatrix(X_test))
y_submit[y_submit < 0] = 1.
sdf['SalePrice'] = y_submit
sdf.to_csv('xg_submission.csv', columns = ['SalePrice'])

In [ ]:
X_train.shape

# Lets do some other work

In [ ]:
df_train = pd.read_csv('../input/train.csv')
df_train = df_train.set_index('Id')

df_test = pd.read_csv('../input/test.csv')
df_test = df_test.set_index('Id')
df_train.head()

In [ ]:
df_train.shape

In [ ]:
price = df_train.SalePrice
df_train = df_train.drop('SalePrice',axis = 1)

In [ ]:
df_train.shape

In [ ]:
df_all = df_train.append(df_test)

In [ ]:
df_all.shape

In [ ]:
numerical = df_all.select_dtypes(include=['int64', 'float64']).copy()
categorical = df_all.select_dtypes(include=['object']).copy()

In [ ]:
categorical.head()

In [ ]:
print(df_all.shape)
print(numerical.shape)
print(categorical.shape)

In [ ]:
# Null values
numerical.isnull().values.sum()
numerical.isnull().sum()

In [ ]:
categorical.isnull().values.sum()
categorical.isnull().sum()

In [ ]:
categorical = pd.get_dummies(categorical)

In [ ]:
print(categorical.shape)
categorical.head()

In [ ]:
categorical.shape
print()

In [ ]:
final = np.hstack((numerical.values, categorical.values))

In [ ]:
print(final.shape)
final[:1]

In [ ]:

print(final.shape)
print(price.shape)
final[:1]


In [ ]:
print(final.shape)
# Impute missing
imp = pp.Imputer(missing_values='NaN', strategy='most_frequent', axis=0)
imp = imp.fit(final)
final = imp.transform(final)
final.shape

In [ ]:
X = final[:1460, :]
Y = price
test = final[1460: , :]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3)

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train, Y_train)

In [ ]:
model.score(X_test, Y_test)

In [ ]:
from sklearn import linear_model

In [ ]:
lasso = linear_model.Lasso(alpha=151.7, max_iter=2000)
lasso.fit(X_train, Y_train)

In [ ]:
lasso.score(X_test, Y_test)

In [ ]:
ridge = linear_model.Ridge(alpha=25.5, max_iter=2000)
ridge.fit(X_train, Y_train)

In [ ]:
ridge.score(X_test, Y_test)

In [ ]:
final.shape

In [ ]:
#PCA
from sklearn.decomposition import PCA

test_n = final.shape[0]

pca = PCA()
pca.fit(final[:test_n,:], price)
final = pca.transform(final)
final.shape

In [ ]:
train.head(3)

In [ ]:
np.unique(train['Street'], return_counts=True)